In [2]:
import os 
import re 
import cantera as ct

directory ='../'
species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

 


def make_vol_frac_list():

    block = \
   '''
To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.01)

gas = ct.Solution(f'./chemkin/dups/dup_chem.cti')
results = {}  

  
#this part calculates the flame calcs for all of the vol_frac list, using the previous vol frac count as a guess for the current one
for i in  range(len(vol_frac_list)):
    try:

        string = f'****************************starting new volume fraction: {vol_frac_list[i]}**************************'
        print(string)
        x = vol_frac_list[i]
        norm_ox = (1-x)*.21

'''
    
    return block

def make_vol_frac_dict(label_list,num):
    '''Makes a volume fraction dictionary, assuming there are equal parts fuel in the blend'''

    vol_frac_dict = \
    '''
        vol_frac_dict = {'''
    l = list(range(len(label_list)-1))
    for i in l:
        vol_frac_dict += f"'{label_list[i]}': (x/{num}/norm_ox), "
    vol_frac_dict += f"'{label_list[-1]}':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox"
    vol_frac_dict += '}'
    
    block2 = \
    '''
        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.max_time_step_count = 900
        loglevel = 1 
        if i!=0:
            d = f'./flame_calcs/chem_CALC/test_{vol_frac_list[i-1]}.csv'
            if os.path.exists(d):  
                arr2 = ct.SolutionArray(gas)
                arr2.read_csv(d)
                flame.set_initial_guess(data=arr2)
                print(' initial guess has been set')
        #"False" stops the calculation from retrying over and over, thanks Chao 
        flame.solve(loglevel=loglevel, auto=False)
        #flame.solve(loglevel=loglevel, auto=True)
        Su = flame.velocity[0]
        results[x] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        df1.to_csv(f'./flame_calcs/chem_CALC/test_{x}.csv', index=False)            
            
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass

vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)

    
with open('final_calcs.csv', 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(vol_fracs)
    writers.writerow(flame_speeds)
        
'''
    vol_frac_dict += block2
    return vol_frac_dict

def make_bash_script(label_list):
    bash_script = f'''#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time=6-00:00:00
#SBATCH --job-name=fc_{'_'.join(label_list[0:-1])}
#SBATCH --error=fc/error/fc.slurm.log
#SBATCH --output=fc/output/fc_output.slurm.log
##SBATCH --cpus-per-task=5
##SBATCH --mem-per-cpu=8Gb
##SBATCH --ntasks=1 
##SBATCH --array=1
#SBATCH --partition=west

source activate cantera_env
python flame_speed_calc_all.py

'''
    return bash_script




In [3]:
def make_blends(list_of_blend_files, num):
    """Proved with a list of blend files, 
    specify the number of fuel species in the blend (num), this will run flame speed calc on all of them.
    
    example: make_blend([list using os.listdir(directory)])
    """
    
    
    flame_speed_calc = '''

import cantera as ct
import numpy as np
import pandas as pd

print("Running Cantera Version: " + str(ct.__version__))

    '''

    
    
    
    species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

    for file in list_of_blend_files: 
  
        #make pattern that re.match will use 
        pattern = ''
        l = list(range(num-1))
        print(l)
        for i in l:
            pattern += '([A-Z0-9]+)_'
        pattern += '([A-Z0-9]+)'
        print(pattern)

        #match and make a list of the species matched in each file
        match = re.match(pattern, file)
        label = []
        l = list(range(num))
        for i in l: 
            label.append(match.group(i+1))
     
        #correct label if it has a funky name
        labels_we_hate={'CH2FCHF2':'C2H3F3','CH2FCH2F':'C2H4F2'}
        for i in range(len(label)):
            if label[i] in labels_we_hate.keys(): 
                label[i]= labels_we_hate[label[i]]
        print(label)
 
        #add oxygen to list, and rename 
        ox_label = 'O2' 
        species_to_look_for = [i for i in label]
        species_to_look_for.append(ox_label)
        print(species_to_look_for)

        #get species name from cti file
        chem_file = f'../{file}/chemkin/dups/dup_chem.cti'
        gas = ct.Solution(chem_file)
        species_names_we_want = {}
        for i in species_to_look_for:
            for name in gas.species_names:
                pattern = f"{i}\([0-9]+\)"
                m = re.match(pattern, name)
                if m:
                    species_names_we_want[i]=name
                    print(i, name)
                    break
                else:
                    print(f"didnt find {i}")
        print(file, species_names_we_want)


        
        #now make the damn thing
        flame_speed_calc += make_vol_frac_list()
        flame_speed_calc += make_vol_frac_dict(list(species_names_we_want.values()),num)
        bash_script = make_bash_script(list(species_names_we_want.values()))
        d = os.path.join(directory,file)
        os.makedirs(d,exist_ok=True)
        with open(os.path.join(d,f'flame_speed_calc.py'),'w') as f: 
            for l in flame_speed_calc: 
                f.write(l)
        with open(os.path.join(d,f'flame_speed_run.sh'),'w') as f: 
            for l in bash_script: 
                f.write(l)
                
        os.makedirs(os.path.join(directory,'flame_calcs'),exist_ok=True)
  



In [5]:
list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
list_of_blends = ['CH2F2_CH3CF3_CH3CHF2']

make_blends(list_of_blends, 3)

[0, 1]
([A-Z0-9]+)_([A-Z0-9]+)_([A-Z0-9]+)
['CH2F2', 'CH3CF3', 'CH3CHF2']
['CH2F2', 'CH3CF3', 'CH3CHF2', 'O2']


CanteraError: 
***********************************************************************
CanteraError thrown by Application::findInputFile:

Input file ../CH2F2_CH3CF3_CH3CHF2/cantera/chem.cti not found in directories 
'.', 
'/home/khalil.nor/.conda/envs/cantera_env/lib/python3.9/site-packages/cantera/data', 
'/home/khalil.nor/.conda/envs/cantera_env/share/cantera/data'

To fix this problem, either:
    a) move the missing files into the local directory;
    b) define environment variable CANTERA_DATA to
         point to the directory containing the file.
***********************************************************************


[0, 1]
([A-Z0-9]+)_([A-Z0-9]+)_([A-Z0-9]+)
['CH2F2', 'C2H5F', 'C2H3F3', 'O2']
[0, 1]
([A-Z0-9]+)_([A-Z0-9]+)_([A-Z0-9]+)
['C2H5F', 'C2H4F2', 'CH3CF3', 'O2']


'\n    mole_frac_dict = {\'***label1***\': (x/norm_ox), \'***label2***\': (x/norm_ox), \'***label4***\': (x/norm_ox), \'***label3***\':((1-x)*.21)/norm_ox, \'N2\':((1-x)*0.79)/norm_ox } \n    gas.TPX = To, Po, mole_frac_dict\n    width = 0.08\n    flame = ct.FreeFlame(gas, width=width)\n    flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) \n    flame.max_time_step_count = 900\n    loglevel = 1 \n    flame.solve(loglevel=loglevel, auto=True)\n    Su = flame.u[0]\n    results[x] = Su\n    sltn = flame.to_solution_array()\n    pd = sltn.to_pandas()\n    pd.to_csv(f\'data_3_21/test_{x}.csv\')\n\nvol_fracs = list(results.keys())\nflame_speeds = list(results.values())\n\n\nprint("volume fractions are:")\nprint(vol_fracs)\n\nprint("flame speeds are:")\nprint(flame_speeds)\n\n\n'


mole_frac_dict = {C2H5F: (x/norm_ox), C2H4F2: (x/norm_ox), CH3CF3: (x/norm_ox), O2:((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}
